In [6]:

import cv2 as cv
import os
import numpy as np
from keras.models import model_from_json
import matplotlib.pyplot as plt
import tensorflow as tf
import time

In [7]:
cam=cv.VideoCapture(0)
minValue=70
interrupt=-1
class_names=['Forward', 'Next', 'Pause', 'Play', 'Previous', 'Reverse', 'Volume Down', 'Volume Up']
if not cam.isOpened():
    print("Error!Camera is not opened.")
    exit()

In [8]:
json_file = open("model-bw.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
loaded_model.load_weights("model-bw.h5")


In [ ]:
while True:
    ret,frame = cam.read()
    frame = cv.flip(frame, 1)
    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
    cv.rectangle(frame, (220-1, 9), (720+1, 519), (255,0,0) ,1)
    roi = frame[10:510, 220:720]
    cv.imshow("Frame", frame)
    roi = cv.resize(roi, (128,128))
    cv.imshow("roi", roi)
    img_array = tf.keras.utils.img_to_array(roi)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = loaded_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    if(np.max(score)>0.5):
        print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
         .format(class_names[np.argmax(score)], 100 * np.max(score))
          )
    interrupt = cv.waitKey(10)
    time.sleep(1)
    if interrupt & 0xFF == 27: # esc key
        break
    
cam.release()
cv.destroyAllWindows()